# Ground-truth generation pipeline (In progress)

Pipeline to generate Lidar or GNSS ground-truth from the prism trajectories gathered by the RTS setup

In [18]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import importlib
import scripts.theodolite_utils as theodo_u
import scripts.groundtruth_utils as theodo_g

In [19]:
theodo_u = importlib.reload(theodo_u)

# Path of the RTS data
path = "../data/20221110/"

# Reading sensor data
# Sensor = "Robosense_32"
Sensor = "Robosense_32"   # GNSS or Robosense_32
path_sensor_file = path+"ICP/icp_odom.csv"

# Sensor = "GNSS"
# path_sensor_file = path+"gps_data/"+"gpsmiddle.txt"
path_sensor_file_synch_time = path+"gps_data/delay_synchronization_GNSS_3.txt"  # If applied to GNSS
Gps_reference_chosen = 3    # 1: front, 2: back, 3: middle   # Only for GNSS

parameters = [
    [1,2,1,1,3],  # 1. Apply filtering or not (Module 1), 2-3-4. Parameters tau_r, tau_a, tau_e (Module 1), 5. Parameter tau_s (Module 2)
]

# Path of the rosbags to process the RTS data
file = [
        # "/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism.bag"
        "/home/maxime/data/FR2023/20221110/2022-11-10-10-43-57.bag"
       ]

output = [
        path
]

In [20]:
theodo_g = importlib.reload(theodo_g)

theodo_g.pipeline_groundtruth(path, Sensor, path_sensor_file, parameters, file, output, path_sensor_file_synch_time, Gps_reference_chosen)

[1, 2, 1, 1, 3]
Number of data for theodolites: [3986 3734 3826]
Bad measures: 3486
3442 3189 3288


100%|██████████| 32/32 [00:00<00:00, 143.89it/s]


Interpolation finished !
Saved !
Conversion done !
Conversion done !
Pipeline done !
